In [ ]:
import numpy as np
import scipy as sci
import scipy.integrate
import matplotlib.pyplot as plt
from matplotlib import animation

%config InlineBackend.figure_format='retina'
%matplotlib osx

from matplotlib.animation import FFMpegWriter
metadata = dict(title='2D animation', artist='Matplotlib')
writer = FFMpegWriter(fps=100, metadata=metadata,bitrate=200000)
fig = plt.figure(dpi=200)

In [ ]:
#  name.      v1.           v2.           T.        set x and y limits to -/+
# II.C-5 0.5512729728  0.5504821832  86.941701946             3
# I.A-4  0.5379557207  0.3414578545  26.918669616            1.5
# I.B-7  0.1862378160  0.5787138661  33.6414187604           1.5
# values from https://arxiv.org/pdf/1705.00527v2.pdf

### CONSTANTS ###
G = 1
T = int(86.941701946)  # orbital period

# Define masses
m1 = 1.0
m2 = 1.0
m3 = 1.0

# Define initial position vectors
r1 = [-1, 0]
r2 = [-r1[0], -r1[1]]
r3 = [0, 0]

# Convert pos vectors to arrays
r1 = np.array(r1,dtype="float64")
r2 = np.array(r2,dtype="float64")
r3 = np.array(r3,dtype="float64")

# Define initial velocities
v1 = [0.5512729728, 0.5504821832]
v2 = v1
v3 = [-2*v1[0], -2*v1[1]]

# Convert velocity vectors to arrays
v1 = np.array(v1,dtype="float64")
v2 = np.array(v2,dtype="float64")
v3 = np.array(v3,dtype="float64")

In [ ]:
def ThreeBodyEquations(w,t,G,m1,m2,m3):
    r1 = w[:2]
    r2 = w[2:4]
    r3 = w[4:6]
    v1 = w[6:8]
    v2 = w[8:10]
    v3 = w[10:12]
    r12 = np.linalg.norm(r2-r1)
    r13 = np.linalg.norm(r3-r1)
    r23 = np.linalg.norm(r3-r2)
    
    dv1bydt = m2*(r2-r1)/r12**3 + m3*(r3-r1)/r13**3
    dv2bydt = m1*(r1-r2)/r12**3 + m3*(r3-r2)/r23**3
    dv3bydt = m1*(r1-r3)/r13**3 + m2*(r2-r3)/r23**3
    dr1bydt = v1
    dr2bydt = v2
    dr3bydt = v3
    
    r_derivs = np.concatenate((dr1bydt,dr2bydt,dr3bydt))
    v_derivs = np.concatenate((dv1bydt,dv2bydt,dv3bydt))
    derivs = np.concatenate((r_derivs,v_derivs))
    
    return derivs

In [ ]:
# Package initial parameters
init_params = np.array([r1,r2,r3,v1,v2,v3])
init_params = init_params.flatten()
time_span = np.linspace(0,T+20,5000)  # run for T+20 and 5000 points

# Run the ODE solver
three_body_sol=sci.integrate.odeint(ThreeBodyEquations,init_params,time_span,args=(G,m1,m2,m3))

In [ ]:
r1_sol=three_body_sol[:,:2]
r2_sol=three_body_sol[:,2:4]
r3_sol=three_body_sol[:,4:6]

In [ ]:
## SAVE AS MP4 ##
fig, ax = plt.subplots()

with writer.saving(fig, "animation_3.mp4", dpi=200):
    for i in range(len(time_span)):
        ax.set_facecolor('white')
        ax.grid(False)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["bottom"].set_visible(False)
        ax.spines["left"].set_visible(False)
        
        ax.clear()
        
        ax.plot(r1_sol[:i,0],r1_sol[:i,1],color="#00C9C8", alpha=0.5)
        ax.plot(r2_sol[:i,0],r2_sol[:i,1],color="#9296F0", alpha=0.5)
        ax.plot(r3_sol[:i,0],r3_sol[:i,1],color="b", alpha=0.5)
        
        ax.scatter(r1_sol[i,0],r1_sol[i,1],color="#00C9C8",marker="o",s= m1*30)
        ax.scatter(r2_sol[i,0],r2_sol[i,1],color="#9296F0",marker="o",s=m2*30)
        ax.scatter(r3_sol[i,0],r3_sol[i,1],color="b",marker="o",s=m3*30)
        
        ax.set_xlim(-3, 3)
        ax.set_ylim(-3, 3)
        
        plt.draw()
        plt.pause(0.01)
        writer.grab_frame()

In [ ]:
## PLOTTING ##
fig, ax = plt.subplots()
ax.set_facecolor('white')
ax.grid(False)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)
        
ax.plot(r1_sol[:,0],r1_sol[:,1],color="#00C9C8", alpha=0.5)
ax.plot(r2_sol[:,0],r2_sol[:,1],color="#9296F0", alpha=0.5)
ax.plot(r3_sol[:,0],r3_sol[:,1],color="b", alpha=0.5)
        
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
plt.show()
plt.savefig('cover.png')